# Start RSI model
https://chatgpt.com/c/67f7d833-ddf4-800a-87be-ea47a130482b

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
# from tensorflow.keras.models import Sequential
from keras.src.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense
from keras.src.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from ta.momentum import RSIIndicator
from sklearn.model_selection import train_test_split

In [ ]:
# Load data
df = pd.read_csv("../src/XAGUSD-197001010000--H1-rates.csv", sep=r'\s+', engine='python')
df['DATETIME'] = pd.to_datetime(df['<DATE>'] + ' ' + df['<TIME>'])
df.set_index('DATETIME', inplace=True)
df = df.drop(columns=['<DATE>', '<TIME>'])

In [ ]:
# Rename for clarity
df.columns = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'TICKVOL', 'VOL', 'SPREAD']

In [ ]:
# Calculate RSI
rsi = RSIIndicator(df['CLOSE'], window=14).rsi()
df['RSI'] = rsi

In [ ]:
# Create labels: 1 = reversal coming, 0 = no reversal
# Rule: RSI crosses 70 downward = SELL, RSI crosses 30 upward = BUY
df['label'] = 0
for i in range(1, len(df)):
    if df['RSI'].iloc[i - 1] > 70 >= df['RSI'].iloc[i]:
        df.loc[df.index[i], 'label'] = 1  # possible SELL
    elif df['RSI'].iloc[i - 1] < 30 <= df['RSI'].iloc[i]:
        df.loc[df.index[i], 'label'] = 1  # possible BUY

df = df.dropna()

In [ ]:
# Normalize features
features = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'TICKVOL', 'VOL', 'SPREAD', 'RSI']
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[features])
df_scaled = pd.DataFrame(df_scaled, columns=features, index=df.index)
df_scaled['label'] = df['label']

In [ ]:
# Create sequences (e.g., past 100 candles as input)
SEQUENCE_LENGTH = 100

def create_sequences(data, seq_len):
    X, y = [], []
    for i in range(len(data) - seq_len):
        seq = data.iloc[i:i+seq_len][features].values
        label = data.iloc[i+seq_len]['label']
        X.append(seq)
        y.append(label)
    return np.array(X), np.array(y)

X, y = create_sequences(df_scaled, SEQUENCE_LENGTH)

In [ ]:
# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
# Model
model = Sequential([
    LSTM(64, return_sequences=False, input_shape=(SEQUENCE_LENGTH, len(features))),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

In [ ]:
# Predict on last 100 candles (real scenario)
last_100 = df_scaled[-SEQUENCE_LENGTH:][features].values.reshape(1, SEQUENCE_LENGTH, len(features))
reversal_prob = model.predict(last_100)[0][0]

if reversal_prob > 0.5:
    print(f"🔁 Trend reversal likely! (probability: {reversal_prob:.2f})")
else:
    print(f"➡️ No reversal expected yet. (probability: {reversal_prob:.2f})")